# Scrape Mangareader

## Set-up for a single page
### Selenium to access JS components

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import requests
import time

In [6]:
# Find the number of chapters in the manga "Vagabond Colored Edition"
# setup webdriver
driver = webdriver.Chrome()

# navigate to the webpage
driver.get('https://mangareader.to/read/dorohedoro-57/en/chapter-1')

# Find the button with the text "Reject All" and click it using JavaScript
button = driver.find_element(By.XPATH, "//span[text()='Reject All']")
driver.execute_script("arguments[0].click();", button)

# Find the element with the class "dropdown-item mode-item" and data-value "horizontal" and click it using JavaScript
element = driver.find_element(By.XPATH, "//a[@class='dropdown-item mode-item' and @data-value='horizontal']")
driver.execute_script("arguments[0].click();", element)

# make the driver wait for 5 seconds
time.sleep(5)

# use BeautifulSoup to parse the HTML
soup = BeautifulSoup(driver.page_source, 'html.parser')

# find the first li element with the class "item reading-item chapter-item"
li_element = soup.find('li', class_='item reading-item chapter-item')

# extract the data-number attribute and convert it to an integer
data_number = float(li_element['data-number'])

In [7]:
import os
# Loop through all the chapters
for chapter in range(1, int(data_number) + 1):
    driver.get(f"https://mangareader.to/read/dorohedoro-57/en/chapter-{chapter}")

    # Update the soup object with the new page content
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all div elements with class "ds-image shuffled loaded" and "ds-image shuffled"
    divs = soup.find_all('div', class_=['ds-image shuffled loaded', 'ds-image shuffled'])

    # If no divs were found, try finding divs with class "ds-image loaded"
    if not divs:
        divs = soup.find_all('div', class_='ds-image loaded')

    # Check if any data URLs were found
    if not divs:
        print(f"Chapter {chapter}: No images found")
        continue  # Skip to the next chapter

    # Extract data-url from each div and store in a list
    data_urls = [div.get('data-url') for div in divs]

    # Download images
    images = []
    for url in data_urls:
        try:
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            images.append(img)
        except requests.exceptions.RequestException as e:
            print(f"Chapter {chapter}: Error downloading image: {e}")

    # Check if any images were downloaded
    if not images:
        print(f"Chapter {chapter}: Failed to download any images")
        continue  # Skip to the next chapter

    # Convert images to JPEG format
    images = [img.convert('RGB') for img in images]

    os.makedirs(f"Chapter_{chapter}", exist_ok=True)

    # Save images in the chapter subfolder  
    for i, img in enumerate(images):
        img.save(f"Chapter_{chapter}/Image_{i+1}.jpeg")

    print(f"Chapter {chapter}: Downloaded {len(images)} images")

Chapter 1: Downloaded 24 images
Chapter 2: Downloaded 26 images
Chapter 3: Downloaded 24 images
Chapter 4: Downloaded 24 images
Chapter 5: Downloaded 24 images
Chapter 6: Downloaded 24 images
Chapter 7: Downloaded 34 images
Chapter 8: Downloaded 28 images
Chapter 9: Downloaded 28 images
Chapter 10: Downloaded 28 images
Chapter 11: Downloaded 28 images
Chapter 12: Downloaded 28 images
Chapter 13: Downloaded 28 images
Chapter 14: Downloaded 23 images
Chapter 15: Downloaded 24 images
Chapter 16: Downloaded 24 images
Chapter 17: Downloaded 24 images
Chapter 18: Downloaded 24 images
Chapter 19: Downloaded 24 images
Chapter 20: Downloaded 24 images
Chapter 21: Downloaded 25 images
Chapter 22: Downloaded 24 images
Chapter 23: Downloaded 24 images
Chapter 24: Downloaded 24 images
Chapter 25: Downloaded 24 images
Chapter 26: Downloaded 23 images
Chapter 27: Downloaded 24 images
Chapter 28: Downloaded 24 images
Chapter 29: Downloaded 24 images
Chapter 30: Downloaded 24 images
Chapter 31: Downloa

## Find bonus chapters

In [8]:
valid_urls = []  # Create an empty list to store valid URLs

for chapter in range(1, int(data_number) + 2):

    # Check for a decimal chapter
    bonus_chapter = chapter + 0.5
    bonus_url = f"https://mangareader.to/read/dorohedoro-57/en/chapter-{bonus_chapter}"

    response = requests.get(bonus_url)

    if response.status_code == 200:
        valid_urls.append(bonus_url)